In [1]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.0 MB/s eta 0:00:00a 0:00:01


In [1]:
import os
import pandas as pd
import nltk

In [2]:
with open('scraped_articles/Netclan20241017.txt', 'r') as file:
    text = file.read()

text

'AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy - Blackcoffer Insights\nClient Background\n\nClient:A leading IT & tech firm in the USA\n\nIndustry Type:IT\n\nProducts & Services:IT Consulting, IT Support, SaaS, Marketing Strategy\n\nOrganization Size:10+\n\nThe Problem\n\nBuilding AI and ML based YouTube analytics and content creation tool that will help youtuber to understand their subscriber’s watching behaviour, help them in content research, creation and publication.\n\nOur Solution\n\nCreated a MERN stack web application and integrated AI models to helps youtuber to generated titles, descriptions, tags, hashtags, captions etc. Help them to check thumbnail quality, analysis on the videos using video auditor tool, analysis on comments using sentiments analysis, help to under their subscribers using churn predication AI model.\n\nSolution Architecture\n\nDeliverables\n\nWeb Applications\n\nSupports\n\nMaintenance

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

# Creating a set of stop words that is removed from the entire corpus of text
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(text)
filtered_text = [w for w in word_tokens if not w in stop_words]
filtered_text = ' '.join(filtered_text)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anuragprasad/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
with open('positive-words.txt', 'r') as file:
    pos_words = file.read().splitlines()

with open('negative-words.txt', 'r') as file:
    neg_words = file.read().splitlines()

keys = ['positive', 'negative']
values = [pos_words, neg_words]
master_dict = dict(zip(keys,values))

In [41]:
pos_score = 0
neg_score = 0
for word in filtered_text.split(' '):
    if word in pos_words:
        pos_score+=1
    elif word in neg_words:
        neg_score+=1

neg_score *= -1
print(pos_score)
print(neg_score)

0
-1


In [46]:
data = pd.read_csv('Input.xlsx - Sheet1.csv')

In [60]:
content = []
for file in os.listdir('scraped_articles'):

    with open(f'scraped_articles/{file}', 'r') as file:
        article = file.read()
    content.append(article)


In [62]:
data['content'] = content

In [63]:
data

,URL_ID,URL,content
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,ROAS Dashboard for Campaign-Wise Google Ads Bu...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,Analyzing the Impact of Positive Emotions and ...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,Enhancing Front-End Features and Functionality...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,Google Local Service Ads Missed Calls and Mess...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,Splitting of Songs into its Vocals and Instrum...
...,...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...,Traction Dashboards of Marketing Campaigns and...
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...,Google LSA API Data Automation and Dashboardin...
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...,Healthcare Data Analysis - Blackcoffer Insight...
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...,Google Local Service Ads (LSA) Data Warehouse ...


In [57]:
df = pd.DataFrame(columns = ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 
                             'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 
                             'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 
                             ' AVG NUMBER OF WORDS PER SENTENCE',
                              'COMPLEX WORD COUNT', 'WORD COUNT',
                              'SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
                              'AVG WORD LENGTH'])